In [8]:
# | default_exp tools.base

In [1]:
# | export
from abc import ABC, abstractmethod
from typing import Dict, Any, Optional
from dataclasses import dataclass
from enum import Enum

In [2]:
# | export
class ToolCategory(Enum):
    """BaseTool categories for better organization"""
    FILESYSTEM = "filesystem"
    SYSTEM = "system"
    ANALYSIS = "analysis"
    INTELLIGENCE = "intelligence"
    PLANNING = "planning"
    UTILITIES = "utilities"
    CUSTOM = "custom"


@dataclass
class ToolMetadata:
    """Metadata for tools"""
    name: str
    description: str
    category: ToolCategory
    version: str = "1.0.0"
    author: Optional[str] = None
    requires_approval: bool = False
    is_dangerous: bool = False


class BaseTool(ABC):
    """Enhanced base class for all tools"""
    
    def __init__(self, metadata: ToolMetadata):
        self.metadata = metadata
        self._validate_metadata()
    
    def _validate_metadata(self):
        """Validate tool metadata"""
        if not self.metadata.name:
            raise ValueError("BaseTool name is required")
        if not self.metadata.description:
            raise ValueError("BaseTool description is required")
    
    @property
    def name(self) -> str:
        """BaseTool name"""
        return self.metadata.name
    
    @property
    def description(self) -> str:
        """BaseTool description"""
        return self.metadata.description
    
    @property
    def category(self) -> ToolCategory:
        """BaseTool category"""
        return self.metadata.category
    
    @abstractmethod
    def get_parameters_schema(self) -> Dict[str, Any]:
        """Get the parameters schema for this tool"""
        pass
    
    @abstractmethod
    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute the tool with given parameters"""
        pass
    
    def validate_parameters(self, parameters: Dict[str, Any]) -> bool:
        """Validate parameters against schema (override for custom validation)"""
        # Basic validation - can be overridden
        return True
    
    def get_openai_schema(self) -> Dict[str, Any]:
        """Get OpenAI-compatible tool schema"""
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": self.get_parameters_schema()
            }
        }
    
    def get_info(self) -> Dict[str, Any]:
        """Get tool information"""
        return {
            "name": self.name,
            "description": self.description,
            "category": self.category.value,
            "version": self.metadata.version,
            "author": self.metadata.author,
            "requires_approval": self.metadata.requires_approval,
            "is_dangerous": self.metadata.is_dangerous,
            "parameters": self.get_parameters_schema()
        }
